In [1]:
!pip install -q --upgrade huggingface_hub transformers datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 799.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 下载数据

In [2]:
from huggingface_hub import hf_hub_download
import zipfile
import os
import shutil

def get_train_zip(name):
  # 下载数据集并且解压到databases文件夹
  dataset_path = hf_hub_download(repo_id="Follow-Lang/set.mm", repo_type="dataset", filename=f"datasets/train/{name}.zip")
  with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall("databases/train/")

def get_folder_size(folder_path):
    total_size = 0
    # os.walk() generates the file names in a directory tree
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            # Join the directory path with the filename to get full file path
            file_path = os.path.join(dirpath, filename)
            # Only add file size if it's a file (skip if it's a symbolic link, etc.)
            if os.path.isfile(file_path):
                total_size += os.path.getsize(file_path) / (1024 * 1024) # Convert bytes to MB
    return total_size

In [3]:

# 删除旧文件夹
if os.path.exists('databases/train'):
  shutil.rmtree('databases/train')

# 下载数据集并且解压到databases文件夹
train_files = ['train_0', 'train_1']

for file in train_files:
  get_train_zip(file)

train_size = get_folder_size("databases/train")

print(f"train data size = {train_size} MB")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train_0.zip:   0%|          | 0.00/95.1M [00:00<?, ?B/s]

train_1.zip:   0%|          | 0.00/120M [00:00<?, ?B/s]

train data size = 3120.514115333557 MB


In [4]:
import os

def get_train_lines(folder_path):
    total_lines = 0
    max_length = 0
    # os.walk() generates the file names in a directory tree
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            # Join the directory path with the filename to get full file path
            file_path = os.path.join(dirpath, filename)
            # Only count lines if it's a file (skip symbolic links, etc.)
            if os.path.isfile(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                      total_lines += 1  # Count the number of lines
                      max_length = max(max_length, len(line.strip().split(' ')))
    return total_lines, max_length


In [5]:
train_lines, max_length = get_train_lines("databases/train")
print(f"Total Train Lines: {train_lines}")
print(f"Max Length: {max_length}")

Total Train Lines: 4194368
Max Length: 2050


In [6]:
from huggingface_hub import hf_hub_download
words_path = hf_hub_download(repo_id="Follow-Lang/set.mm", repo_type="dataset", filename=f"datasets/train/words.txt")
with open(words_path, 'r') as f:
  words = ['<bos>', '<eos>', '<pad>'] + [line.strip() for line in f.readlines()]
word_map = {word: idx for idx, word in enumerate(words)}
print(len(words))
bos = word_map['<bos>']
eos = word_map['<eos>']
pad = word_map['<pad>']
end_of_state = word_map['</state>']
end_of_action = word_map['</action>']
ignore_id = -100

def encode_ids(s: str) -> list[int]:
    return [word_map.get(word, eos) for word in s.split(' ')]

def decode_ids(ids: list[int]) -> str:
    return ' '.join([words[i] if i >= 0 else "<pad>" for i in ids])

datasets/train/words.txt:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

2578


# 生成数据集

In [7]:
import os
import json
import random
from tqdm.notebook import tqdm  # 使用 notebook 友好的进度条

max_length = 1024

def save_to_jsonl(data, filename):
    """ 将数据保存到 JSONL 文件 """
    with open(filename, "a") as f:
        for entry in data:
            json.dump(entry, f)
            f.write("\n")

def handle_line(line):
    """ 处理单行文本，生成状态和动作的 token 以及成本 """
    if len(line.strip()) == 0:
        return None, None, None, None
    try:
        costs, words = line.strip().split('\t')
        costs = [int(word) for word in costs.split(' ')]

        toks = encode_ids(words)
        state_index = toks.index(end_of_state)
        action_index = toks.index(end_of_action)

        state_toks = toks[1:state_index]  # 截去 <state> 和 </state>
        action_toks = toks[state_index+2:action_index]
        next_state_toks = toks[action_index+2:-1] if len(toks)-(action_index+2) >= 3 else toks[action_index+2:]

        if any(len(t) > max_length or len(t) == 0 for t in [state_toks, action_toks, next_state_toks]):
            return None, None, None, None
        return state_toks, action_toks, next_state_toks, costs
    except ValueError:
        return None, None, None, None

def generate_data_from_files(folder_path, output_file):
    """ 从指定文件夹读取 .txt 文件，处理并存储为 JSONL 格式 """
    if os.path.exists(output_file):
        os.remove(output_file)

    txt_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".txt")]
    random.shuffle(txt_files)

    with tqdm(total=train_lines, desc="Processing files") as pbar:
        for file_name in txt_files:
            batch_data = []
            with open(file_name, 'r', encoding='utf-8') as f:
                for line in f:
                    pbar.update(1)
                    state_toks, action_toks, next_state_toks, costs = handle_line(line)
                    if state_toks is None:
                        continue
                    # 追加3条数据：原状态、动作、下一个状态
                    batch_data.append({"cost": costs[0], "state": state_toks, "action": action_toks, "next_state": next_state_toks})
                    batch_data.append({"cost": costs[1], "state": action_toks, "action": None, "next_state": None})
                    batch_data.append({"cost": costs[2], "state": next_state_toks, "action": None, "next_state": None})

            # 批量写入 JSONL 文件
            save_to_jsonl(batch_data, output_file)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "databases/train"
output_file = "/content/drive/MyDrive/train_data.jsonl"
if not os.path.os.path.exists(output_file):
  generate_data_from_files(folder_path, output_file)


Processing files:   0%|          | 0/4194368 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

output_file = "/content/drive/MyDrive/train_data.jsonl"
dataset = load_dataset("json", data_files=output_file, split="train")
n = 10
sampled_data = dataset.shuffle(seed=42).select(range(n))
for i, sample in enumerate(sampled_data):
    print(f"{i} cost   : {sample['cost']}")
    print(f"  state  : {decode_ids(sample['state'])}")
    print(f"  aciton : {decode_ids(sample['action']) if sample['action'] else None}")
    print(f"  nstate : {decode_ids(sample['next_state']) if sample['next_state'] else None}")

Generating train split: 0 examples [00:00, ? examples/s]

0 cost   : 87
  state  : |- wi wceq cv gs0 cv gs1 wb wo wceq cv gs0 vc0 wceq cv gs0 vc1 wo wceq cv gs1 vc0 wceq cv gs1 vc1 |- wi wceq cv gs1 cun vc0 vc1 wb wo wceq cv gs1 vc0 wceq cv gs1 vc1 wo wceq cun vc0 vc1 vc0 wceq cun vc0 vc1 vc1
  aciton : |- wi wceq cv gs1 cun vc0 vc1 wb wo wceq cv gs1 vc0 wceq cv gs1 vc1 wo wceq cun vc0 vc1 vc0 wceq cun vc0 vc1 vc1 -| wi wceq cv gs1 cun vc0 vc1 wb wceq cv gs1 vc0 wceq cun vc0 vc1 vc0 -| wi wceq cv gs1 cun vc0 vc1 wb wceq cv gs1 vc1 wceq cun vc0 vc1 vc1
  nstate : |- wi wceq cv gs0 cv gs1 wb wo wceq cv gs0 vc0 wceq cv gs0 vc1 wo wceq cv gs1 vc0 wceq cv gs1 vc1 |- wi wceq cv gs1 cun vc0 vc1 wb wceq cv gs1 vc0 wceq cun vc0 vc1 vc0 |- wi wceq cv gs1 cun vc0 vc1 wb wceq cv gs1 vc1 wceq cun vc0 vc1 vc1
1 cost   : 125
  state  : |- wnfc vs0 cop cab gs0 wal vs0 wcel cv gs0 csb cv gs1 vs0 vc3 cab gs0 wal vs0 wcel cv gs0 vc5 |- wnf vs0 wnfc vs0 csb cv gs1 vs0 vc3 -| wi wtru wnfc vs0 csb cv gs1 vs0 vc3 -| wi wtru wnfc vs0 vc5
  aciton : |- wnf vs0 wnfc v

# 构建奖励模型

In [1]:
from transformers import GPT2LMHeadModel
from transformers.modeling_outputs import ModelOutput
import torch.nn as nn
import torch
from dataclasses import dataclass
from typing import Optional, Tuple

from safetensors.torch import load_file  # Import the load function for safetensors

max_length = 1024

def custom_collate_fn(batch):
    def pad_toks(tok_batch):
      if tok_batch is None or len(tok_batch) == 0:
        return None, None
      max_len = max(len(item) if item else 0 for item in tok_batch)
      max_len = min(max_len, max_length)  # 限制到 max_length
      states = [torch.tensor(item + [pad] * (max_len - len(item))) if item else torch.tensor([pad] * max_len) for item in tok_batch]
      state_masks = [torch.tensor([1] * len(item) + [0] * (max_len - len(item))) if item else torch.tensor([0] * max_len) for item in tok_batch]
      return torch.stack(states), torch.stack(state_masks)

    # 其他字段类似处理
    costs = torch.tensor([item["cost"] for item in batch], dtype=torch.float32).unsqueeze(-1)
    states, states_mask = pad_toks([item["state"] for item in batch])
    actions, actions_mask = pad_toks([item["action"] for item in batch])
    nstates, nstates_mask = pad_toks([item["next_state"] for item in batch])

    return {
        "costs": costs,
        "states": states,
        "states_mask": states_mask,
        "actions": actions,
        "actions_mask": actions_mask,
        "nstates": nstates,
        "nstates_mask": nstates_mask
    }

@dataclass
class CustomModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None
    costs: Optional[torch.FloatTensor] = None

class FollowCostModel(GPT2LMHeadModel):
    def __init__(self, config):
        config.output_hidden_states=True
        super().__init__(config)
        self.costs_head = nn.Linear(config.n_embd, 1) # 新增 cost head，用于预测奖励值

    def forward(self, input_ids, attention_mask=None, **kwargs):
        # 获取标准输出
        outputs = super().forward(input_ids, attention_mask=attention_mask, **kwargs)
        hidden_states = outputs.hidden_states[-1]

        costs_score = self.costs_head((hidden_states * attention_mask.unsqueeze(-1)).sum(dim=1))  # 取最后一个 token 的状态计算奖励
        costs_mask = attention_mask.sum(axis=-1).clamp(0, 1).unsqueeze(-1)  # 应用 mask

        # 返回 BaseModelOutputWithPastAndCrossAttentions 以包含 cost_score
        return CustomModelOutput(
            loss=outputs.loss,
            logits=outputs.logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            costs=costs_score * costs_mask  # Custom costs output
        )

    @classmethod
    def from_pretrained(cls, checkpoint_path, config=None, *model_args, **kwargs):
        if config is None:
            config = AutoConfig.from_pretrained(checkpoint_path)

        model = cls(config)

        # Load weights from safetensors file
        model_weights = load_file(os.path.join(checkpoint_path, "model.safetensors"))
        model.load_state_dict(model_weights, strict=False)  # Load the model weights

        return model

# 训练

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import random
import numpy as np
import torch

# 生成一个随机种子
seed = random.randint(0, 2**32 - 1)

# 设置各个随机数生成器的种子
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# 输出种子，便于复现
print(f"Random seed set to: {seed}")

Random seed set to: 2954722319


In [4]:
import wandb

# 初始化 wandb，恢复之前的 run
wandb_project = "follow-reward-gpt2"
wandb_run_id = "hidden_mean-02"
wandb.init(project=wandb_project, resume="allow", id=wandb_run_id)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: colorlessboy (colorlessboy-zhejiang-university). Use `wandb login --relogin` to force relogin


In [5]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download

words_path = hf_hub_download(repo_id="Follow-Lang/set.mm", repo_type="dataset", filename=f"datasets/train/words.txt")
with open(words_path, 'r') as f:
  words = ['<bos>', '<eos>', '<pad>'] + [line.strip() for line in f.readlines()]
word_map = {word: idx for idx, word in enumerate(words)}
print(len(words))
bos = word_map['<bos>']
eos = word_map['<eos>']
pad = word_map['<pad>']
end_of_state = word_map['</state>']
end_of_action = word_map['</action>']
ignore_id = -100

def encode_ids(s: str) -> list[int]:
    return [word_map.get(word, eos) for word in s.split(' ')]

def decode_ids(ids: list[int]) -> str:
    return ' '.join([words[i] if i >= 0 else "<pad>" for i in ids])

folder_path = "databases/train"
output_file = "/content/drive/MyDrive/train_data.jsonl"
dataset = load_dataset("json", data_files=output_file, split="train")
# 打乱数据集
dataset = dataset.shuffle(seed=seed)  # seed 参数可以固定随机化顺序

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2578


In [6]:
from transformers import AutoConfig, TrainingArguments, Trainer
import torch
import os


# 判断是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 指定模型保存的路径
model_name_or_path = "gpt2"
output_dir = f"/content/drive/MyDrive/{model_name_or_path}-reward"

# 确保 output_dir 文件夹存在
os.makedirs(output_dir, exist_ok=True)

# 查找是否有保存的检查点
checkpoints = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint")]
if checkpoints:
    latest_checkpoint = max(checkpoints, key=os.path.getctime)  # 按创建时间排序选择最新的 checkpoint
    print(f"Using checkpoint: {latest_checkpoint}")
    model = FollowCostModel.from_pretrained(latest_checkpoint, ignore_mismatched_sizes=True)
else:
    latest_checkpoint = None
    print(f"No checkpoint found, using gpt2")
    # 加载自定义模型
    config = AutoConfig.from_pretrained('gpt2', bos_token_id=eos, eos_token_id=eos, pad_token_id=eos, vocab_size=4096)
    model = FollowCostModel(config=config)

batch_size = 8
gradient_accumulation_steps = 32
num_train_epochs = 1

training_args = TrainingArguments(
    output_dir=output_dir,  # 保存模型到指定路径
    per_device_train_batch_size=batch_size,  # 单个设备的batch size
    logging_dir="./logs",  # 日志保存路径
    logging_steps=10,  # 每 10 步记录一次loss
    evaluation_strategy="no",  # 禁用验证
    save_strategy="steps",  # 每隔一段时间保存模型
    save_steps=100,  # 每 10 步保存一次
    save_total_limit=3,  # 最多保留3个保存点
    load_best_model_at_end=False,  # 不需要在训练结束时加载最好的模型
    learning_rate=1e-5,  # 初始学习率
    lr_scheduler_type="cosine",  # 使用余弦学习率调度器
    warmup_steps=100,  # warmup 步数为 100
    fp16=True,  # 启用混合精度训练
    resume_from_checkpoint=latest_checkpoint if latest_checkpoint else None,  # 从检查点恢复
    report_to="wandb",  # 禁用wandb和其他报告工具
    gradient_accumulation_steps=gradient_accumulation_steps,  # 累计 16 个batch 的梯度
    weight_decay=0.1,  # 使用 weight decay 防止过拟合
    adam_beta1=0.9,  # Adam 优化器的 beta1 参数
    adam_beta2=0.95,  # Adam 优化器的 beta2 参数
    max_grad_norm=1.0,  # 梯度裁剪，防止梯度爆炸
    run_name=wandb_project,
    num_train_epochs=num_train_epochs,
    remove_unused_columns=False  # Keep custom columns in the dataset
)

class CustomTrainer(Trainer):
    def __init__(self, k=0.5, lm=0.99, reward_scale=1.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = torch.nn.MSELoss()
        self.k = k
        self.lm = lm
        self.reward_scale = reward_scale
        self.loss_scale = 1.0 / self.args.gradient_accumulation_steps

    def compute_loss(self, model, inputs, return_outputs=False, *args, **kwargs):
        # 计算模型输出
        outputs = model(input_ids=inputs['states'], attention_mask=inputs['states_mask'])
        # 计算标准损失
        loss = self.loss_fct(outputs.costs, self.reward_scale*torch.pow(self.lm, inputs['costs']))
        with torch.no_grad():
            actions_cost = model(input_ids=inputs['actions'], attention_mask=inputs['actions_mask']).costs
            nstates_cost = model(input_ids=inputs['nstates'], attention_mask=inputs['nstates_mask']).costs
        loss = (1 - self.k) * loss + self.k * self.loss_fct(outputs.costs, torch.min(actions_cost, self.lm*nstates_cost))
        loss *= self.loss_scale
        return (loss, outputs) if return_outputs else loss

# 加载和配置 trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=custom_collate_fn,
    k=0.2,
    lm=0.995,
    reward_scale=100.0
)


Using device: cuda
No checkpoint found, using gpt2


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# 继续训练
trainer.train(resume_from_checkpoint=True if latest_checkpoint else False)

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,2977.900200
20,2676.518600
30,1432.869700
40,1227.565400
50,1613.149400
60,1490.475900
70,908.045000
80,1083.677400
90,741.206100
100,635.287500


# 查看预测结果

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import torch

# 生成一个随机种子
seed = random.randint(0, 2**32 - 1)

# 设置各个随机数生成器的种子
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# 输出种子，便于复现
print(f"Random seed set to: {seed}")


Random seed set to: 10579067


In [ ]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download

words_path = hf_hub_download(repo_id="Follow-Lang/set.mm", repo_type="dataset", filename=f"datasets/train/words.txt")
with open(words_path, 'r') as f:
  words = ['<bos>', '<eos>', '<pad>'] + [line.strip() for line in f.readlines()]
word_map = {word: idx for idx, word in enumerate(words)}
print(len(words))
bos = word_map['<bos>']
eos = word_map['<eos>']
pad = word_map['<pad>']
end_of_state = word_map['</state>']
end_of_action = word_map['</action>']
ignore_id = -100

def encode_ids(s: str) -> list[int]:
    return [word_map.get(word, eos) for word in s.split(' ')]

def decode_ids(ids: list[int]) -> str:
    return ' '.join([words[i] if i >= 0 else "<pad>" for i in ids])

folder_path = "databases/train"
output_file = "/content/drive/MyDrive/train_data.jsonl"
dataset = load_dataset("json", data_files=output_file, split="train")
# 打乱数据集
dataset = dataset.shuffle(seed=seed)  # seed 参数可以固定随机化顺序

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


datasets/train/words.txt:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

2578


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoConfig, TrainingArguments, Trainer
import torch
import os


# 判断是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 指定模型保存的路径
model_name_or_path = "gpt2"
output_dir = f"/content/drive/MyDrive/{model_name_or_path}-reward"

# 确保 output_dir 文件夹存在
os.makedirs(output_dir, exist_ok=True)

# 查找是否有保存的检查点
checkpoints = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint")]
if checkpoints:
    latest_checkpoint = max(checkpoints, key=os.path.getctime)  # 按创建时间排序选择最新的 checkpoint
    print(f"Using checkpoint: {latest_checkpoint}")
    model = FollowCostModel.from_pretrained(latest_checkpoint, ignore_mismatched_sizes=True)
else:
    latest_checkpoint = None
    print(f"No checkpoint found, using gpt2")
    # 加载自定义模型
    config = AutoConfig.from_pretrained('gpt2', bos_token_id=eos, eos_token_id=eos, pad_token_id=eos, vocab_size=4096, output_hidden_states=True)
    model = FollowCostModel(config=config)

Using device: cpu
Using checkpoint: /content/drive/MyDrive/gpt2-reward/checkpoint-2500


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


In [ ]:
# 采样 n 条数据
n = 24  # Or any desired number of samples
sampled_data = dataset.select(range(n))
sampled_data = custom_collate_fn(sampled_data)
model.eval()
with torch.no_grad():
    output = model(input_ids=sampled_data['states'].to(device), attention_mask=sampled_data['states_mask'].to(device))
for i, (predict, target) in enumerate(zip(sampled_data['costs'], torch.log(output.costs / 100)/np.log(0.995))):
    print(f"{i} predict={predict} target={target}")

0 predict=tensor([112.]) target=tensor([98.6919])
1 predict=tensor([111.]) target=tensor([137.8156])
2 predict=tensor([16.]) target=tensor([19.2965])
3 predict=tensor([47.]) target=tensor([42.8845])
4 predict=tensor([17.]) target=tensor([26.6215])
5 predict=tensor([105.]) target=tensor([111.4417])
6 predict=tensor([104.]) target=tensor([110.2854])
7 predict=tensor([94.]) target=tensor([78.7754])
8 predict=tensor([27.]) target=tensor([21.8575])
9 predict=tensor([344.]) target=tensor([319.0616])
10 predict=tensor([228.]) target=tensor([131.3599])
11 predict=tensor([125.]) target=tensor([129.7545])
12 predict=tensor([4.]) target=tensor([20.1296])
13 predict=tensor([712.]) target=tensor([991.1899])
14 predict=tensor([92.]) target=tensor([88.6009])
15 predict=tensor([13.]) target=tensor([22.8070])
16 predict=tensor([81.]) target=tensor([84.2256])
17 predict=tensor([313.]) target=tensor([280.6622])
18 predict=tensor([3.]) target=tensor([22.3011])
19 predict=tensor([81.]) target=tensor([80.23